In [12]:
!python3 -m venv env
!source env/bin/activate
!python3 -m pip install pandas numpy openpyxl


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [18]:
import pandas as pd
import numpy as np
import os
# from vavtools import vavtools as vav
db_name = 'ELECTRONICS_SHOP'
data_folder = 'tables'
all_files = os.listdir(data_folder)    
tables = list(filter(lambda f: f.endswith('.xlsx'), all_files))

tables = [
    'order_status.xlsx', 
    'country.xlsx',
    'category.xlsx', 
    'product.xlsx', 
    'payment_status.xlsx',
    'customer.xlsx', 
    'address.xlsx',
    'order.xlsx',
    'order_item.xlsx',
    'payment.xlsx',
    'review.xlsx',
    'vat.xlsx'
    ]

In [19]:
alter_tables =  '''

ALTER TABLE `order`
ADD CONSTRAINT `fk_order_1`
  FOREIGN KEY (`STATUS_ID`)
  REFERENCES `ELECTRONICS_SHOP`.`order_status` (`ID`)
  ON DELETE RESTRICT
  ON UPDATE CASCADE;

ALTER TABLE `order`
ADD CONSTRAINT `fk_order_2`
  FOREIGN KEY (`CUSTOMER_ID`)
  REFERENCES `customer` (`ID`)
  ON DELETE RESTRICT
  ON UPDATE CASCADE;

ALTER TABLE `order`
ADD CONSTRAINT `fk_order_3`
  FOREIGN KEY (`ADDRESS_ID`)
  REFERENCES `address` (`ID`)
  ON DELETE RESTRICT
  ON UPDATE CASCADE;

ALTER TABLE `address`
ADD CONSTRAINT `fk_address_2`
  FOREIGN KEY (`COUNTRY_ID`)
  REFERENCES `country` (`ID`)
  ON DELETE RESTRICT
  ON UPDATE CASCADE;

ALTER TABLE `payment`
ADD CONSTRAINT `fk_payment_1`
  FOREIGN KEY (`STATUS_ID`)
  REFERENCES `payment_status` (`ID`)
  ON DELETE RESTRICT
  ON UPDATE CASCADE;

ALTER TABLE `payment`
ADD CONSTRAINT `fk_payment_2`
  FOREIGN KEY (`ORDER_ID`)
  REFERENCES `order` (`ID`)
  ON DELETE RESTRICT
  ON UPDATE CASCADE;

ALTER TABLE `order_item`
ADD CONSTRAINT `fk_order_item_1`
  FOREIGN KEY (`PRODUCT_ID`)
  REFERENCES `product` (`ID`)
  ON DELETE RESTRICT
  ON UPDATE CASCADE;

ALTER TABLE `order_item`
ADD CONSTRAINT `fk_order_item_2`
  FOREIGN KEY (`ORDER_ID`)
  REFERENCES `order` (`ID`)
  ON DELETE RESTRICT
  ON UPDATE CASCADE;

ALTER TABLE `category`
ADD CONSTRAINT `fk_category_1`
  FOREIGN KEY (`PARENT_CATEGORY_ID`)
  REFERENCES `category` (`ID`)
  ON DELETE NO ACTION 
  ON UPDATE NO ACTION;
  
ALTER TABLE `product`
ADD CONSTRAINT `fk_product_1`
  FOREIGN KEY (`CATEGORY_ID`)
  REFERENCES `category` (`ID`)
  ON DELETE RESTRICT
  ON UPDATE CASCADE;

ALTER TABLE `review`
ADD CONSTRAINT `fk_review_1`
  FOREIGN KEY (`ORDER_ITEM_ID`)
  REFERENCES `order_item` (`ID`)
  ON DELETE RESTRICT
  ON UPDATE CASCADE;

ALTER TABLE `vat`
ADD CONSTRAINT `fk_vat_1`
  FOREIGN KEY (`CATEGORY_ID`)
  REFERENCES `category` (`ID`)
  ON DELETE RESTRICT
  ON UPDATE CASCADE;
  
'''

In [20]:
class sql_table_creator:
    def __init__(self, table_name, frame, primary_key):
        self.name = table_name
        self.frame = frame
        self.pk = primary_key
        
    def type_selector(self, dtype, col_name):
        dtype = str(dtype)
        if 'DATE' in col_name:
            return 'date'
        if ('ID' in col_name) or ('int' in dtype):
            return 'INTEGER'
        if dtype == 'object':
            return 'VARCHAR(64)'
        if 'float' in dtype:
            return 'DECIMAL(6,0)'
        
    def create_table(self):
        sql_command = f"DROP TABLE IF EXISTS `{self.name}`;\nCREATE TABLE IF NOT EXISTS `{self.name}` ("
        data_types = self.frame.dtypes
        for col in self.frame.columns:
            name = col.upper()
            d_type = self.type_selector(data_types[col], name)
            nullable_status = 'NOT NULL' if 'ID'in col else 'DEFAULT NULL'
            line = f"\n\t`{name}` {d_type} {nullable_status},"
            sql_command += line
        sql_command+=f"\n\tPRIMARY KEY (`{self.pk}`)\n);\n\n"
        return sql_command


class sql_table_filler:
    def __init__(self, table_name, frame):
        self.name = table_name
        self.frame = frame
        
    def fill_table(self):
        for col in self.frame.columns:
            if 'DATE' in col:
                self.frame[col] = pd.to_datetime(self.frame[col]).dt.strftime('%Y-%m-%d')

        sql_command = f"INSERT INTO `{self.name}` (`{'`, `'.join([_.upper() for _ in list(self.frame.columns)])}`) VALUES\n"
        for row in list(self.frame.itertuples(index=False, name=None)):
            sql_command += f"{str(row)},\n"
        return sql_command[:-2] + ';\n\n'


In [21]:
command = f"CREATE DATABASE IF NOT EXISTS `{db_name}` DEFAULT CHARACTER SET = 'utf8' DEFAULT COLLATE 'utf8_general_ci'; \n\n USE {db_name};\nSET FOREIGN_KEY_CHECKS=0;\n\n"
for file in tables:
    df = pd.read_excel(f'{data_folder}/{file}', engine='openpyxl')
    table = sql_table_creator(
        table_name = file.replace('.xlsx',''), 
        frame = df, 
        primary_key = 'ID')
    sql_command = table.create_table()
    command+=sql_command

command+=alter_tables

In [22]:
for file in tables:
    df = pd.read_excel(f'{data_folder}/{file}')
    sql_command = sql_table_filler(file.replace('.xlsx',''), df).fill_table()
    command += sql_command

with open('db_generator.sql', 'w') as f:
    command = command.replace('nan', 'NULL').replace('.0,',',')
    f.write(command)